<a href="https://colab.research.google.com/github/LH-1169213/Equipo3/blob/main/MNA_IAyAA_semana_10_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 10**
###**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Nombres y matrículas de los integrantes del equipo:**

*   Samara García González A01273001
*   Laura Elena Hernández Mata A01169213
*   Evelyn Aylin Rendon Medina A01748750
*   Emmanuel González Calitl A01320739
*   Carlos Enrique Diaz Ayala A01793816


En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [77]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD

#**Ejercicio-1.** 

In [78]:
#rating_final.csv
df_ratingfinal = pd.read_csv("https://raw.githubusercontent.com/LH-1169213/Equipo3/main/rating_final.csv", header = None)

#Renombrando las columnas
df_ratingfinal.columns = df_ratingfinal.iloc[0]
df_ratingfinal.drop(0, axis=0, inplace=True)
df_ratingfinal.head()

,userID,placeID,rating,food_rating,service_rating
1,U1077,135085,2,2,2
2,U1077,135038,2,2,1
3,U1077,132825,2,2,2
4,U1077,135060,1,2,2
5,U1068,135104,1,1,2


In [79]:
#Obteniendo datos de valor del archivo rating_final.csv
ID_rating = df_ratingfinal[['userID','rating']]
ID_rating.head(2)

,userID,rating
1,U1077,2
2,U1077,2


In [80]:
#file is not valid UTF-8. A common workaround is to force a different encoding, commonly 'latin-1'
#Se esta primera solución para el encode porque nos permite visualizar correctamente los acentos
u'\xe9'.encode('latin-1')

#geoplaces2.csv
df_geoplaces = pd.read_csv("https://raw.githubusercontent.com/LH-1169213/Equipo3/main/geoplaces2.csv", header = None) #, encoding='latin-1')

#Renombrando las columnas
df_geoplaces.columns = df_geoplaces.iloc[0]
df_geoplaces.drop(0, axis=0, inplace=True)
df_geoplaces.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
1,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
2,132825,22.1473922,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
3,135106,22.1497088,-100.9760928,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
4,132667,23.7526973,-99.1633594,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
5,132613,23.7529035,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


In [81]:
#Obteniendo datos de valor del archivo geoplaces2.csv
nombres = df_geoplaces[['name']]
nombres['userID'] = ID_rating['userID']
nombres.head(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,name,userID
1,Kiku Cuernavaca,U1077
2,puesto de tacos,U1077


In [82]:
#Tabla relacional
tablaRelacion = pd.merge(ID_rating, nombres, on='userID')
tablaRelacion.head()

,userID,rating,name
0,U1077,2,Kiku Cuernavaca
1,U1077,2,puesto de tacos
2,U1077,2,El Rincón de San Francisco
3,U1077,2,little pizza Emilio Portes Gil
4,U1077,2,Kiku Cuernavaca


In [83]:
#Matriz de utilidad
matriz = tablaRelacion.pivot_table(values='rating', index='userID', columns='name', fill_value=0)
print('Tamaño de la matriz de utilidad:', matriz.shape)
print('número de filas:',matriz.shape[0])
print('número de columnas:',matriz.shape[1])

Tamaño de la matriz de utilidad: (27, 129)
número de filas: 27
número de columnas: 129


In [84]:
matriz.head(10)

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1012,0.0,0.0,0.0,0,0.0,0.0,0,0,2444.4,0.0,...,0,0,0.0,0.0,0.0,0,0.0,0,0.0,0.0
U1015,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,...,0,0,0.0,0.0,0.0,0,0.0,0,0.0,0.0
U1017,0.0,0.0,0.0,0,0.0,0.0,0,40,0.0,0.0,...,0,0,0.0,0.0,0.0,0,0.0,0,0.0,0.0
U1021,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,...,0,0,0.0,0.0,0.0,0,0.0,0,0.0,0.0
U1023,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,...,0,0,0.0,0.0,0.0,0,0.0,0,0.0,0.0
U1026,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,...,0,0,0.0,4200.4,0.0,0,0.0,0,0.0,0.0
U1030,0.0,0.0,0.0,2762500,0.0,0.0,2762500,0,0.0,0.0,...,0,0,0.0,0.0,0.0,0,0.0,2762500,0.0,0.0
U1031,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,...,0,0,0.0,0.0,0.0,0,0.0,0,0.0,0.0
U1044,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0,...,0,0,0.0,0.0,0.0,0,0.0,0,0.0,0.0


In [85]:
print('Total de elementos de la matriz de utilidad: %d' % (matriz.size))
print('Total de elementos diferentes de cero: %d' % (np.count_nonzero(matriz)))
print('Porcentaje de elementos diferentes de cero: %.1f%%' % (100 * np.count_nonzero(matriz) / matriz.size))

Total de elementos de la matriz de utilidad: 3483
Total de elementos diferentes de cero: 127
Porcentaje de elementos diferentes de cero: 3.6%


#**Ejercicio-2.**

In [86]:
#Transpuesta de la matriz de utilidad
#Para obtener la información latente en relación a los lugares de comida
matrizT = matriz.T
print('Tamaño de la matriz de utilidad:', matriz.shape)
print('número de filas:',matrizT.shape[0])
print('número de columnas:',matrizT.shape[1])

Tamaño de la matriz de utilidad: (27, 129)
número de filas: 129
número de columnas: 27


In [87]:
#El número de componentes debe ser menor que el número de características
SVD = TruncatedSVD(n_components=matrizT.shape[1]-1, random_state=0)  
SVD.fit(matrizT)

num_sv = len(SVD.singular_values_)

print('Cantidad de información simplificada con los %d vectores singulares obtenidos:' % num_sv)
print('%.1f%%' %  (100 * (1- (SVD.singular_values_[0:num_sv]).sum() / (SVD.singular_values_).sum())))

Cantidad de información simplificada con los 26 vectores singulares obtenidos:
0.0%


#**Ejercicio-3.**

In [88]:
None

#**Ejercicio-4.**

In [89]:
None

#**Ejercicio-5.**

In [90]:
None

#**Ejercicio-6.**

In [91]:
None

#**Ejercicio-7.**

In [92]:
None

#**Ejercicio-8.**

In [93]:
None

###**Fin de la Actividad de la semana 10 - Sistemas de Recomendación y Reducción de Dimensionalidad con SVD.**